# Window functions

General Elections were held in the UK in 2015 and 2017. Every citizen votes in a constituency. The candidate who gains the most votes becomes MP for that constituency.

All these results are recorded in a table ge

yr	| firstName	| lastName	| constituency	| party	| votes
---:|-----------|-----------|---------------|-------|------:
2015	| Ian	| Murray	| S14000024	| Labour	| 19293
2015	| Neil	| Hay	| S14000024	| Scottish National Party	| 16656
2015	| Miles	| Briggs	| S14000024	| Conservative | 8626
2015	| Phyl	| Meyer	| S14000024	| Green	| 2090
2015	| Pramod	| Subbaraman	| S14000024	| Liberal Democrat	| 1823
2015	| Paul	| Marshall	| S14000024	| UK Independence Party	 | 601
2015	| Colin	| Fox	| S14000024	| Scottish Socialist Party	| 197
2017	| Ian	| MURRAY	| S14000024	| Labour	| 26269
2017	| Jim	| EADIE	| S14000024	| SNP	| 10755
2017	| Stephanie Jane Harley	| SMITH	| S14000024	| Conservative	| 9428
2017	| Alan Christopher	| BEAL	| S14000024	| Liberal Democrats	| 1388

In [1]:
# Prerequesites
from pyhive import hive
%load_ext sql
%sql hive://cloudera@quickstart.cloudera:10000/sqlzoo
%config SqlMagic.displaylimit = 20

## 1. Warming up

Show the **lastName, party** and **votes** for the **constituency** 'S14000024' in 2017.

In [2]:
%%sql
SELECT lastName, party, votes
  FROM ge
    WHERE constituency = 'S14000024' AND yr = 2017
    ORDER BY votes DESC

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


lastname,party,votes
MURRAY,Labour,26269
EADIE,SNP,10755
SMITH,Conservative,9428
BEAL,Liberal Democrats,1388


## 2. Who won?

You can use the RANK function to see the order of the candidates. If you RANK using (ORDER BY votes DESC) then the candidate with the most votes has rank 1.

**Show the party and RANK for constituency S14000024 in 2017. List the output by party**

In [3]:
%%sql
SELECT party, votes,
       RANK() OVER (ORDER BY votes DESC) posn
    FROM ge
    WHERE constituency = 'S14000024' AND yr = 2017
    ORDER BY party

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


party,votes,posn
Conservative,9428,3
Labour,26269,1
Liberal Democrats,1388,4
SNP,10755,2


## 3. PARTITION BY

The 2015 election is a different PARTITION to the 2017 election. We only care about the order of votes for each year.

**Use PARTITION to show the ranking of each party in S14000021 in each year. Include yr, party, votes and ranking (the party with the most votes is 1).**

In [4]:
%%sql
SELECT yr, party, votes,
      RANK() OVER (PARTITION BY yr ORDER BY votes DESC) posn
    FROM ge
    WHERE constituency = 'S14000021'
    ORDER BY party, yr

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


yr,party,votes,posn
2015,Conservative,12465,3
2017,Conservative,21496,1
2019,Conservative,19451,2
2015,Labour,19295,2
2017,Labour,14346,2
2019,Labour,6855,3
2015,Liberal Democrats,1069,4
2017,Liberal Democrats,1112,3
2019,Liberal Democrats,4174,4
2015,SNP,23013,1


## 4. Edinburgh Constituency

Edinburgh constituencies are numbered S14000021 to S14000026.

**Use PARTITION BY constituency to show the ranking of each party in Edinburgh in 2017. Order your results so the winners are shown first, then ordered by constituency.**

In [5]:
%%sql
SELECT constituency, party, votes,
       RANK() OVER (PARTITION BY constituency ORDER BY votes DESC) posn
    FROM ge
    WHERE constituency BETWEEN 'S14000021' AND 'S14000026'
    AND yr  = 2017
    ORDER BY posn, constituency

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


constituency,party,votes,posn
S14000021,Conservative,21496,1
S14000022,SNP,18509,1
S14000023,SNP,19243,1
S14000024,Labour,26269,1
S14000025,SNP,17575,1
S14000026,Liberal Democrats,18108,1
S14000021,Labour,14346,2
S14000022,Labour,15084,2
S14000023,Labour,17618,2
S14000024,SNP,10755,2


## 5. Winners Only

You can use [SELECT within SELECT](https://sqlzoo.net/wiki/SELECT_within_SELECT_Tutorial) to pick out only the winners in Edinburgh.

**Show the parties that won for each Edinburgh constituency in 2017.**

In [6]:
%%sql
SELECT constituency, party FROM
 (SELECT constituency, party, 
  RANK() OVER (PARTITION BY constituency ORDER BY votes DESC) posn
  FROM ge
  WHERE constituency BETWEEN 'S14000021' AND 'S14000026'
  AND yr  = 2017) AS a
    WHERE a.posn=1

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


constituency,party
S14000021,Conservative
S14000022,SNP
S14000023,SNP
S14000024,Labour
S14000025,SNP
S14000026,Liberal Democrats


## 6. Scottish seats

You can use **COUNT** and **GROUP BY** to see how each party did in Scotland. Scottish constituencies start with 'S'

**Show how many seats for each party in Scotland in 2017.**

In [7]:
%%sql
SELECT party, COUNT(party) seats FROM
 (SELECT party, constituency, 
  RANK() OVER (PARTITION BY constituency ORDER BY votes DESC) posn
  FROM ge
  WHERE constituency LIKE 'S%' AND yr=2017) AS a
    WHERE a.posn=1
    GROUP BY party

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


party,seats
Conservative,12
Labour,9
Liberal Democrats,4
SNP,34
